# 🧪 BioAuth: Run `RUN.sh` equivalent in Google Colab (no Docker)

Colab doesn't support running Docker containers, so this notebook compiles and runs the project **natively** using CMake and Ubuntu packages. It mirrors what `RUN.sh` does by building and launching the experiment binaries directly.

In [ ]:
%%bash
set -euo pipefail
echo "Updating apt and installing build deps..."
sudo apt-get update -y
sudo apt-get install -y build-essential cmake git libeigen3-dev libboost-all-dev \
    libssl-dev libgmp-dev libgmpxx4ldbl libpthread-stubs0-dev iproute2 net-tools libtbb-dev
echo "Done. Versions:"
gcc --version | head -n1
cmake --version | head -n1
openssl version

## Get the code
Pick **one** option:
1) **Upload the ZIP** (e.g., *BioAuth-main.zip*) to Colab (left sidebar → Files), then set `ZIP_PATH` and run.
2) (Default) **Git clone** the repository by setting `GIT_URL`.


In [ ]:
import os, shutil, zipfile

# Option A: point this to your uploaded zip (e.g., "/content/BioAuth-main.zip")
ZIP_PATH = "/content/BioAuth-main.zip"  # change if you uploaded elsewhere; leave as-is if not using zip

# Option B: repo URL (if public GitHub). Leave empty if using the ZIP option.
GIT_URL = "https://github.com/MaJinWakeUp/BioAuth.git" #"https://github.com/sakurasfy/BioAuth.git"

WORKDIR = "/content"
TARGET_DIR = os.path.join(WORKDIR, "BioAuth")

# Clean previous copies
if os.path.isdir(TARGET_DIR):
    shutil.rmtree(TARGET_DIR)

if os.path.exists(ZIP_PATH):
    print(f"Unzipping {ZIP_PATH} → {WORKDIR} ...")
    with zipfile.ZipFile(ZIP_PATH) as z:
        z.extractall(WORKDIR)
    # Try to normalize folder name
    # Common patterns: BioAuth-main/, or root folder named after repo
    # If you see a different name, feel free to just set TARGET_DIR accordingly.
    guess_dirs = [
        os.path.join(WORKDIR, d) for d in os.listdir(WORKDIR)
        if os.path.isdir(os.path.join(WORKDIR, d)) and d.lower().startswith("bioauth")
    ]
    if guess_dirs:
        src_dir = guess_dirs[0]
        print("Found directory:", src_dir)
        os.rename(src_dir, TARGET_DIR)
    else:
        raise RuntimeError("Couldn't find unzipped BioAuth directory; set TARGET_DIR manually.")
elif GIT_URL:
    import subprocess
    print("Cloning repo...")
    subprocess.check_call(["git", "clone", "--depth", "1", GIT_URL, TARGET_DIR])
else:
    raise RuntimeError("Please upload a ZIP to /content and set ZIP_PATH, or set GIT_URL to clone.")

print("Ready. Project folder:", TARGET_DIR)
# print("Contents:", os.listdir(TARGET_DIR)[:50])


## Configure & build (CMake)

In [ ]:
%%bash
set -euo pipefail
cd /content/BioAuth
rm -rf build
mkdir -p build
cd build
cmake .. -DCMAKE_BUILD_TYPE=Release
make -j 4
cd ..
echo -e "\033[32mSuccess! Built targets under: /content/BioAuth/build \033[0m"
# echo "Listing experiment binaries (if any):"
# find build/experiments -maxdepth 2 -type f -executable -printf "%p\n" || true


## Run Experiment 1: Secure Authentication
Settings: Vector Length: 1024, Database Size: 512

This will start `party_1` in the background and then run `party_0` in the foreground. Logs are saved under `EXP_DIR`.




In [ ]:
%%bash
set -euo pipefail

# Change this if your path differs:
EXP_DIR="/content/BioAuth/build/experiments/secure-authentication"
cd "$EXP_DIR"

mkdir -p fake-offline-data keys

# Sanity checks
BIN0=./secure_authentication_party_0
BIN1=./secure_authentication_party_1
for b in "$BIN0" "$BIN1"; do
  if [[ ! -x "$b" ]]; then
    echo "Error: expected binary not found or not executable: $b" >&2
    echo "Tip: re-run your CMake build, then re-check this path." >&2
    exit 1
  fi
done

echo "BioAuth Network Simulation Test"
echo "Testing 4 network configurations..."
echo ""

# Start party_1 in background, capture its PID; then run party_0 in foreground
"$BIN1" > party1.log 2>&1 &
party1_pid=$!
sleep 3
"$BIN0" | tee party0.log

# Wait for background party_1 to finish
wait "$party1_pid"

echo
echo "=== Logs saved in: $EXP_DIR ==="
ls -l party*.log || true

# check logs if you want
# echo "Last 20 lines of each log:"
# for f in party1.log party0.log; do
#   echo "---- $f ----"
#   tail -n 20 "$f" || true
# done


## Run Experiment 2: Secure Dot Product Online
Settings: Vector Length: 1024, Database Size: 512




In [ ]:
%%bash
set -euo pipefail

# Adjust this if your path differs:
EXP_DIR="/content/BioAuth/build/experiments/dot-product-db"

cd "$EXP_DIR"

mkdir -p fake-offline-data keys

# Binaries (update names if your build uses different targets)
BIN1=./dot_product_db_party_1
BIN0=./dot_product_db_party_0

# Sanity checks (fail early if not found/executable)
for b in "$BIN1" "$BIN0"; do
  if [[ ! -x "$b" ]]; then
    echo "Error: expected binary not found or not executable: $b" >&2
    echo "Tip: re-run CMake build and verify the output path." >&2
    exit 1
  fi
done

# Equivalent of the Docker CMD script
echo "BioAuth Dot Product Database Test"
echo "Testing dot product database operations..."
echo ""

# Start party_1 in the background; run party_0 in the foreground
# (Logs captured to files for convenience)
"$BIN1" > party1.log 2>&1 &
party1_pid=$!
sleep 3
"$BIN0" | tee party0.log

# Wait for background party_1 to finish
wait "$party1_pid"

echo
echo "=== Logs saved in: $EXP_DIR ==="
ls -l party*.log || true

# check logs if you want
# echo "Last 20 lines of each log:"
# for f in party1.log party0.log; do
#   echo "---- $f ----"
#   tail -n 20 "$f" || true
# done


## Run Experiment 3: Secure Dot Product Simulated Offline
Settings: Vector Length: 1024, Database Size: 256




In [ ]:
%%bash
set -euo pipefail

# Adjust this if your path differs:
EXP_DIR="/content/BioAuth/build/experiments/dot-product-db"

cd "$EXP_DIR"

# Equivalent of: RUN mkdir -p fake-offline-data keys
mkdir -p fake-offline-data keys

# Binaries for the OFFLINE variant
BIN1=./dot_product_db_offline_party_1
BIN0=./dot_product_db_offline_party_0

# Sanity checks (fail early if not found/executable)
for b in "$BIN1" "$BIN0"; do
  if [[ ! -x "$b" ]]; then
    echo "Error: expected binary not found or not executable: $b" >&2
    echo "Tip: re-run the CMake build and verify the output path." >&2
    exit 1
  fi
done

# Equivalent of the Docker CMD script
echo "BioAuth Dot Product Database Offline Test"
echo "Testing dot product database offline operations..."
echo ""

# Start party_1 in the background; run party_0 in the foreground
"$BIN1" > party1.log 2>&1 &
party1_pid=$!
sleep 3
"$BIN0" | tee party0.log

# Wait for background party_1 to finish
wait "$party1_pid"

echo
echo "=== Logs saved in: $EXP_DIR ==="
ls -l party*.log || true

# check logs if you want
# echo "Last 20 lines of each log:"
# for f in party1.log party0.log; do
#   echo "---- $f ----"
#   tail -n 20 "$f" || true
# done


## Run Experiment 4: Secure Comparison
Settings: Rounds: 512




In [ ]:
%%bash
set -euo pipefail

# Adjust this if your path differs:
EXP_DIR="/content/BioAuth/build/experiments/secure-com"

cd "$EXP_DIR"

# Equivalent of: RUN mkdir -p fake-offline-data keys
mkdir -p fake-offline-data keys

# Binaries
BIN1=./com_party_1
BIN0=./com_party_0

# Sanity checks
for b in "$BIN1" "$BIN0"; do
  if [[ ! -x "$b" ]]; then
    echo "Error: expected binary not found or not executable: $b" >&2
    echo "Tip: re-run your CMake build and verify EXP_DIR." >&2
    exit 1
  fi
done

# Equivalent of the Docker CMD script
echo "BioAuth Secure Comparison Test"
echo "Testing secure comparison operations..."
echo ""

# Start party_1 (server) in background with log file
"$BIN1" > party1_output.log 2>&1 &
party1_pid=$!

# Give server time to start, then run party_0 (client)
sleep 3
"$BIN0" > party0_output.log 2>&1

# Wait for party_1 to finish
wait "$party1_pid"

# Print results like your CMD
echo ""
echo "=== Party 1 (Server) Results ==="
cat party1_output.log || true
echo ""
echo "=== Party 0 (Client) Results ==="
cat party0_output.log || true
echo ""
echo "=== Secure Comparison Tests Completed ==="

# Bonus: show where logs are
echo ""
echo "Logs saved in: $EXP_DIR"
ls -l party*_output.log || true

# check logs if you want
# echo "Last 20 lines of each log:"
# for f in party1_output.log party0_output.log; do
#   echo "---- $f ----"
#   tail -n 20 "$f" || true
# done